- 首先读取零售业的采购数据，对数据进行重划分以减少分区数量，最后将数据缓存起来以便快速访问

In [ ]:
df = spark.read.format("csv")\
.option("header", "true")\
.option("inferSchema", "true")\
.load("./data/retail-data/all/*.csv")\
.coalesce(5)

df.cache()
df.createOrReplaceTempView("dfTable")

- 基本的聚合操作将作用于整个DataFrame，最简单的例子是count方法

In [ ]:
df.count() == 541909

- count操作是动作操作，会立即返回计算结果
- 不仅可以用count获得数据集的总体大小
- 还可以缓存整个DataFrame到内存里

<h4>聚合函数</h4>

- 第一个聚合函数是count，count聚合操作是一个transformation转化操作而不是动作操作
- 我们可以对指定列进行计数
- 也可以使用count\(*)或count(1)对所有列进行计数

In [2]:
from pyspark.sql.functions import count

In [ ]:
df.select(count("StockCode")).show()

-- in SQL<br>
SELECT COUNT\(*) FROM dfTable

<h4>countDistinct</h4>

- 获得唯一组的数量
- 仅在针对某列计数时才有意义

In [3]:
from pyspark.sql.functions import countDistinct

In [ ]:
df.select(countDistinct("StockCode")).show()

-- in SQL<br>
SELECT COUNT\(DISTINCT *) FROM dfTable

<h4>approx_count_distinct</h4>

- 在处理大数据集时，精确的统计计数并不重要，某种精度的近似值也可以接受

In [4]:
from pyspark.sql.functions import approx_count_distinct

In [ ]:
df.select(approx_count_distinct("StockCode", 0.1)).show()

<h4>first和last</h4>

- 基于DataFrame中行的顺序，这两个函数可以得到DataFrame的第一个值和最后一个值

In [ ]:
from pyspark.sql.functions import first, last
df.select(first("StockCode"), last("StockCode")).show()

-- in SQL<br>
SELECT first(StockCode), last(StockCode) FROM dfTable

<h4>min和max</h4>

- 若要从DataFrame中提取最小值和最大数值，使用min和max函数

In [1]:
from pyspark.sql.functions import min, max

In [ ]:
df.select(min("Quantity"), max("Quantity")).show()

-- in SQL<br>
SELECT min(Quantity), max(Quantity) FROM dfTable

<h4>sum</h4>

- 使用sum函数累加一列的所有值

In [2]:
from pyspark.sql.functions import sum

In [ ]:
df.select(sum("Quantity")).show()

-- in SQL<br>
SELECT sum(Quantity) FROM dfTable

<h4>sumDistinct</h4>

- 对一组去重值进行求和

In [4]:
from pyspark.sql.functions import sumDistinct

In [ ]:
df.select(sumDistinct("Quantity")).show()

-- in SQL<br>
SELECT SUM(Quantity) FROM dfTable

<h4>avg</h4>

- Spark提供avg函数获取平均值

In [8]:
from pyspark.sql.functions import sum, count, avg, expr

In [ ]:
df.select(
    count("Quantity").alias("total_transactions"),
    sum("Quantity").alias("total_purhcases"),
    avg("Quantity").alias("avg_purchases"),
    expr("mean(Quantity)").alias("mean_purchases")
)\
.selectExpr(
    "total_purchases/total_transactions",
    "avg_purchases",
    "mean_purchases"
).show()

<h4>方差和标准差</h4>

- 方差是各数据样本与均值之间差的平方的平均值
- 在Spark中，如果使用variance函数和stddev函数，默认计算样本方差和样本标准差

In [10]:
from pyspark.sql.functions import var_pop, stddev_pop
from pyspark.sql.functions import var_samp, stddev_samp

In [ ]:
df.select(var_pop("Quantity"), var_samp("Quantity"), stddev_pop("Quantity"), stddev_samp("Quantity"))

-- in SQL<br>
SELECT var_pop(Quantity), var_samp("Quantity"), stddev_pop("Quantity"), stddev_samp("Quantity") FROM dfTable

<h4>skewness和kurtosis</h4>

- 偏度系数：衡量数据相对于平均值的不对称程度
- 峰度系数：衡量数据分布形态的陡缓程度

In [11]:
from pyspark.sql.functions import skewness, kurtosis

In [ ]:
df.select(skewness("Quantity"), kurtosis("Quantity")).show()

-- in SQL<br>
SELECT skewness("Quantity"), kurtosis("Quantity") FROM dfTable

<h4>协方差和相关性</h4>

- 有的函数可以比较量两列值之间的相互关系
- 其中两个函数就是cov和corr，它们分别用于计算协方差和相关性，相关性采用Pearson相关系数，范围是-1~1，协方差的范围由输入数据决定
- 其中协方差分为总体协方差和样本协方差

In [13]:
from pyspark.sql.functions import corr, covar_pop, covar_samp

In [ ]:
df.select(corr("InvoiceNo","Quantity"), covar_samp("InvoiceNo","Quantity"), covar_pop("InoviceNo", "Quantity")).show()

-- in SQL<br>
SELECT corr("InvoiceNo","Quantity"), covar_samp("InvoiceNo","Quantity"), covar_pop("InoviceNo", "Quantity") FROM dfTable

<h4>聚合输出复杂类型</h4>

- 收集某列上的值到一个list列表里
- 或者将unique唯一值收集到一个set集合里

In [1]:
from pyspark.sql.functions import collect_set, collect_list

In [2]:
df.agg(collect_set("Country"), collect_list("Country"))

NameError: name 'df' is not defined

-- in SQL<br>
SELECT collect_set(Country), collect_list(Country) FROM dfTable

<h4>分组</h4>

- 之前的聚合操作都是DataFrame级别的操作，或者针对某一列的函数
- 更常见的任务是根据分组数据计算，典型应用是处理类别数据


- 分组操作分为两个阶段
    1. 首先指定对其进行分组的一列或多列
    2. 然后指定一个或多个聚合操作

In [ ]:
df.groupBy("InvoiceNo", "CustomerId").count().show()

-- in SQL<br>
SELECT count\(*) FROM dfTable GROUP BY InvoiceNo, CustomrId

<h4>使用表达式分组</h4>

- 通常在agg中指定聚合操作，还可以对操作结果重命名

In [1]:
df.groupBy("InvoiceNo").agg(
    count("Quantity").alias("quan"),
    expr("count(Quantity)")
).show()

NameError: name 'df' is not defined

<h4>使用Map进行分组</h4>

- 即在agg方法里使用map指定每列的聚合方法|

In [3]:
df.groupBy("InvoiceNo").agg(expr("avg(Quantity)"), expr("stddev_pop(Quantity)")).show()

NameError: name 'df' is not defined

-- in SQL<br>
SELECT avg(Quantity), stddev_pop(Quantity), InvoiceNo FROM dfTable<br>
GROUP BY InvoiceNo

<h4>window函数</h4>

<h4>首先添加一个date列，将发票日期转换为仅包含日期信息，不包括时间信息的列</h4>

In [1]:
from pyspark.sql.functions import col, to_date

In [ ]:
dfWithDate = df.withColumn("date", to_date(col("InvoiceDate"), "MM/d/yyyy H:mm"))
dfWithDate.createOrReplaceTempView("dfWithDate")

<h4>分组集</h4>

- 对应到SQL的grouping sets
- 分组集是用于将多个聚合操作组合在一起的工具
- 以下先解释什么是分组集

<h4>首先去除包含空值的行</h4>

In [ ]:
dfNoNull = dfWithDate.drop()
dfNoNull.createOrReplaceTempView("dfNoNull")

- 如果我们想聚合客户id和股票代码，比如统计每位客户每个股票的数量总和，以下两种方式都是可以的

-- in SQL<br>
SELECT CustomerId, stockCode, sum(Quantity) FROM dfNoNull<br>
GROUP BY CustomerId, stockCode<br>
ORDER BY CustomerId DESC, stockCode DESC

-- in SQL<br>
SELECT CustomerId, stockCode, sum(Quantity) FROM dfNoNull<br>
GROUP BY CustomerId, stockCode GROUPING SETS((CustomerId, stockCode))

- 上述两种写法效果一样
- 但如果我还想只聚合客户，以得到股票数量总和，只靠group by是做不到将两种分组统计合并的，需要使用union，将两种分组结果合并
- 而grouping sets会统计所有种类的分组情况

-- in SQL<br>
SELECT CustomerId, stockCode, sum(Quantity) FROM dfNoNull<br>
GROUP BY CustomerId, stockCode<br>
UNION<br>
SELECT CustomerId, sum(Quantity) FROM dfNoNull<br>
GROUP BY CustomerId

- GROUPING SETS操作仅在SQL中可用
- 若想在DataFrame中执行相同的操作，使用rollup和cube操作可以得到完全相同的结果

<h4>rollup</h4>

- rollup分组聚合是一种多维聚合操作，可以执行多种group-by风格的计算
- 以下创建日期，国家分组集，并使用求和的分组操作
- 结果将包含：所有日期所有国家的股票总数，每个日期所有国家的股票总数，每个日期每个国家的股票总数
- 每列的null值表示不区分该列的总数，比如Country列为null表示该日期所有地点总数
- 如果两列都为null值，则表示所有日期和地点总数

In [ ]:
rolledUpDF = dfNoNull.rollup("Date","Country").agg(sum("Quantity"))\
.selectExpr("Date","Country","sum(Quantity)" as total_quantity)\
.orderBy("Date")
rolledUpDF.show()

In [ ]:
rolledUpDF.where("Country IS NULL").show()
rolledUpDF.where("Date IS NULL").show()

<h4>cube</h4>

- cube分组聚合更进一步，不用于rollup的分级聚合
- 它会对所有参与的列值进行所有维度的全组合聚合
- 也就是相比于rollup，cube会对每个国家不同日期的股票计算总和

In [3]:
from pyspark.sql.functions import sum

In [ ]:
dfNoNull.cube("Date","Country").agg(sum(col("Quantity")))\
.select("Date", "Country", "sum(Quantity)").orderBy("Date").show()

<h4>对元数组进行分组</h4>

- 当使用cube和rollup时，希望能查询聚合级别
- 在此之前，我们使用的是df.where("someColumn is null")
- 现在可以使用grouping_id来完成此操作

In [4]:
from pyspark.sql.functions import grouping_id, sum, expr

In [ ]:
dfNoNull.cube("CustomerID", "StockCode").agg(grouping_id(), sum("Quantity"))\
.orderBy(expr("grouping_id()"), ascending=False).show()

<h4>透视转换</h4>

- 透视转换可以根据某列的不同行创建多个列
- 例如我们有Country列，那么就可以对每个Country执行聚合操作
- 具体来说就是，使用了透视转换后，DF会为每一个Country和数值型列组合产生一个新列
- 例如，对于CN，就有CN_sum(Quantity),CN_sum(UnitPrice),CN_sum(CustomerID)，每个数值型列都会和聚合列的每个种类产生一个新列

In [ ]:
pivoted = dfWithDate.groupBy("date").pivot("Country").sum()

In [ ]:
pivoted.where("date > '2011-12-05'").select("date", "USA_sum(Quantity)").show()